https://colab.research.google.com/drive/1P9iQHtUH5KUbTVtp8B4-AopZzEEPE0lw?usp=sharing#scrollTo=4YS7XI2bZTyz

https://github.com/aub-mind/arabert

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip -q install optuna==2.3.0
!pip -q install transformers==4.2.1
!pip -q install farasapy
!pip -q install pyarabic
# !git clone https://huggingface.co/qarib/bert-base-qarib

     |████████████████████████████████| 258 kB 7.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 80 kB 10.3 MB/s 
     |████████████████████████████████| 164 kB 75.8 MB/s 
     |████████████████████████████████| 75 kB 5.7 MB/s 
     |████████████████████████████████| 111 kB 78.0 MB/s 
     |████████████████████████████████| 49 kB 7.3 MB/s 
     |████████████████████████████████| 141 kB 76.5 MB/s 
     |████████████████████████████████| 1.8 MB 8.0 MB/s 
     |████████████████████████████████| 895 kB 54.6 MB/s 
     |████████████████████████████████| 2.9 MB 58.0 MB/s 
     |████████████████████████████████| 126 kB 8.0 MB/s 


In [ ]:
import pandas as pd
import numpy as np

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

In [ ]:
hard_map = { 0 : 'fake', 1 : 'real', }

df_train = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataText/PPV2train_news.csv", header=0,encoding='utf-8')
df_train.label= df_train.label.apply(lambda x: hard_map[x])

df_val = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataText/PPV2val_news.csv", header=0,encoding='utf-8')
df_val.label= df_val.label.apply(lambda x: hard_map[x])

label_list_HARD = ['fake', 'real']


# Trainer

In [ ]:
# from arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score

from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures


In [ ]:
dataset_name = 'NEWS'
model_name = 'qarib/bert-base-qarib60_1970k'
task_name = 'classification'
max_len = 500

In [ ]:
# arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])

# df_train.text = df_train.text.apply(lambda x:   arabert_prep.preprocess(x))
# df_val.text = df_val.text.apply(lambda x:   arabert_prep.preprocess(x))  

In [ ]:
class BERTDataset():
    def __init__(self, text=None, target= None, model_name = None, max_len= None, label_map= None):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

# 

In [ ]:
label_map = { v:index for index, v in enumerate(label_list_HARD) }
print(label_map)
train_dataset = BERTDataset(df_train.text.to_list(),df_train.label.to_list(),model_name, max_len,label_map)
val_dataset  = BERTDataset(df_val.text.to_list()  ,df_val.label.to_list()  ,model_name,  max_len, label_map)


{'fake': 0, 'real': 1}


In [ ]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [ ]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds))
  print(confusion_matrix(p.label_ids,preds))

  # macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  # c_repoert= classification_report(p.label_ids,preds))

  return {
      'macro_f1' : macro_f1,
      # 'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc,
      # 'classification_report' :c_repoert
  }

# Regular Training

In [ ]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.fp16 = True
# training_args.per_device_train_batch_size =16
# training_args.per_device_eval_batch_size = 16

training_args.per_device_train_batch_size = 16  
training_args.per_device_eval_batch_size = 16


training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8

steps_per_epoch = (len(df_train)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 42
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

160
1280


In [ ]:
trainer = Trainer(
    model = model_init(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
) 

Some weights of the model checkpoint at qarib/bert-base-qarib60_1970k were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were 

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
0,No log,0.062378,0.970993,0.969416,0.972594,0.978939,23.929700,53.573000
1,No log,0.055518,0.978585,0.975431,0.981841,0.984399,24.073800,53.253000
2,No log,0.079087,0.979401,0.982191,0.976684,0.985179,24.108600,53.176000
3,0.074900,0.078283,0.980508,0.982733,0.978329,0.985959,23.953700,53.520000
4,0.074900,0.085648,0.982713,0.983819,0.981619,0.987520,23.925200,53.584000
5,0.074900,0.100556,0.980372,0.986112,0.974929,0.985959,23.990500,53.438000
6,0.001900,0.097819,0.981484,0.986635,0.976573,0.986739,23.942500,53.545000
7,0.001900,0.097837,0.981484,0.986635,0.976573,0.986739,24.035400,53.338000


              precision    recall  f1-score   support

           0       0.95      0.96      0.96       304
           1       0.99      0.98      0.99       978

    accuracy                           0.98      1282
   macro avg       0.97      0.97      0.97      1282
weighted avg       0.98      0.98      0.98      1282

[[292  12]
 [ 15 963]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       304
           1       0.99      0.99      0.99       978

    accuracy                           0.98      1282
   macro avg       0.98      0.98      0.98      1282
weighted avg       0.98      0.98      0.98      1282

[[297   7]
 [ 13 965]]
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       304
           1       0.99      0.99      0.99       978

    accuracy                           0.99      1282
   macro avg       0.98      0.98      0.98      1282
weighted avg       0.99     

TrainOutput(global_step=1280, training_loss=0.030032311379909516, metrics={'train_runtime': 2171.6582, 'train_samples_per_second': 0.589, 'total_flos': 16635459840336000, 'epoch': 8.0})

In [ ]:
trainer.evaluate(val_dataset)

              precision    recall  f1-score   support

           0       0.99      0.96      0.97       304
           1       0.99      1.00      0.99       978

    accuracy                           0.99      1282
   macro avg       0.99      0.98      0.98      1282
weighted avg       0.99      0.99      0.99      1282

[[291  13]
 [  4 974]]


{'epoch': 8.0,
 'eval_accuracy': 0.9867394695787831,
 'eval_loss': 0.09783701598644257,
 'eval_macro_f1': 0.9814839830591274,
 'eval_macro_precision': 0.9866347260144592,
 'eval_macro_recall': 0.9765734312775805,
 'eval_runtime': 23.9835,
 'eval_samples_per_second': 53.453}

# Test

In [ ]:
df_test = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/DataText/PPV2test_news.csv")
print(df_test.label.value_counts())


1    1198
0     405
Name: label, dtype: int64


In [ ]:
df_test.label= df_test.label.apply(lambda x: hard_map[x])

In [ ]:
test_dataset = BERTDataset(df_test.text.to_list(),df_test.label.to_list(),model_name, max_len,label_map)

In [ ]:
trainer.evaluate(test_dataset)

              precision    recall  f1-score   support

           0       0.99      0.96      0.97       405
           1       0.99      1.00      0.99      1198

    accuracy                           0.99      1603
   macro avg       0.99      0.98      0.98      1603
weighted avg       0.99      0.99      0.99      1603

[[ 389   16]
 [   4 1194]]


{'epoch': 8.0,
 'eval_accuracy': 0.9875233936369308,
 'eval_loss': 0.09714117646217346,
 'eval_macro_f1': 0.9833158477589322,
 'eval_macro_precision': 0.9882993712278931,
 'eval_macro_recall': 0.978577464498444,
 'eval_runtime': 29.9765,
 'eval_samples_per_second': 53.475}

# Predict

In [ ]:
class Dataset():
    def __init__(self, text=None, target= None, model_name = None, max_len= None, label_map= None):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask)

In [ ]:

title_example = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/Data/title_example.csv")
title_example.shape

count=0
for title in title_example.text:
   test_dataset = Dataset([title],None,model_name, max_len,label_map)
   y_label,_,_ = trainer.predict(test_dataset)
   preds = np.argmax(y_label, axis=1)   
   print(preds)
   if preds==1 :
    count=count+1

    
print("Real : "+str(count))

[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
[0]
Real : 0


In [ ]:
text_example = pd.read_csv("drive/My Drive/Arabic_FakeNews/Model/Balance/Data/text_example.csv")
text_example.shape
count=0

for text in text_example.text:
   text_dataset = Dataset([text],None,model_name, max_len,label_map)
   y_label,_,_ = trainer.predict(text_dataset)
   preds = np.argmax(y_label, axis=1)   
   print(preds)
   if preds==1 :
    count=count+1
    
print("Real : "+str(count))

[1]
[0]
[1]
[0]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[1]
[0]
[1]
[1]
[1]
[1]
[1]
[1]
Real : 17


In [ ]:
# # Tokenize test data
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512) 
# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized) 
# # Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2) 
# # Define test trainer
# test_trainer = Trainer(model) 
# # Make prediction
# raw_pred, _, _ = test_trainer.predict(test_dataset) 
# # Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)

In [ ]:
# print("===> Predicting...")
# trainer1 = Trainer(
#     model= model_init(),
#     args=training_args,
#     eval_dataset=test_dataset
# )
        
# results = {}
# result = trainer1.predict(test_dataset)

In [ ]:
#  ----- 3. Predict -----#
# # Load test data
# test_data = pd.("drive/My Drive/Arabic_FakeNews/Model/Data/test_news.csv", header=0,encoding='utf-8')
# X_test = list(test_data["text"])
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# # Create torch dataset
# test_dataset = Dataset(X_test_tokenized)

# # Load trained model
# model_path = "output/checkpoint-50000"
# model = BertForSequenceClassification.from_pretrained(model_path, num_labels=2)

# # Define test trainer
# test_trainer = Trainer(model)

# # Make prediction
# raw_pred, _, _ = test_trainer.predict(test_dataset)

# # Preprocess raw predictions
# y_pred = np.argmax(raw_pred, axis=1)